In [1]:
import pandas as pd
import sys
sys.path.append('../src')
from data_helpers import load_data_and_labels
from preprocess import load_data
from preprocess import create_vocabulary, vectorize_features
from preprocess import embedding_words_with_pretrained_model
from tensorflow.contrib import learn
import pickle

/Users/chi/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# Read Data

In [2]:
tweets = load_data()

In [3]:
tweets, tweets_y = load_data_and_labels(tweets)

Split sentences: : 100%|██████████| 51088/51088 [00:54<00:00, 932.48it/s] 


In [4]:
tweets_vec, vocab_processor = vectorize_features(tweets)

Instructions for updating:
Please use tensorflow/transform or tf.data.
Instructions for updating:
Please use tensorflow/transform or tf.data.
Instructions for updating:
Please use tensorflow/transform or tf.data.


In [5]:
vocab_processor

In [6]:
vocab_dict = vocab_processor.vocabulary_._mapping

In [7]:
pre_embedding = embedding_words_with_pretrained_model(vocab_dict)

Embedding words by pretrained model: : 100%|██████████| 28002/28002 [00:00<00:00, 69000.69it/s]


In [8]:
tweets.head(10)

,Unnamed: 0,id,genre,statusid,pn,p,n,neu,unk,content,words
0,3,10028,10000,522408394871672832,0,0,0,1,0.0,ｴｸｽﾍﾟﾘｱZ3がよさそうねー\n,ｴｸｽﾍﾟﾘｱ z 3 よい さ そう
1,5,10030,10000,522408539814260736,0,1,0,0,0.0,Xperia Z3の純正walkmanアプリはalac対応するみたいだね 音楽周りは本当GO...,xperia z 3 純正 walkman アプリ alac 対応 する みたい だ 音楽 ...
2,7,10032,10000,522409063154339840,0,0,0,1,0.0,少し時間空いちゃいましたが、Xperia Z3のカメラ機能について、ちょっとだけですけどまと...,少し 時間 空く いる ます た xperia z 3 カメラ 機能 ちょっと です まとめ...
3,8,10033,10000,522409073057091584,0,0,0,0,1.0,日向「研磨おたおめー。これプレゼント!!」 孤爪「こ、これは」 日向「ビビった?」 孤爪「X...,日向 研磨 た め ー これ プレゼント !!」 孤 爪 こ ビビる ?」 xperia e...
4,13,10038,10000,522411170183913472,0,1,0,0,0.0,#Xperia Z2にしてから、スケジュールストリートに登録する時にいちいち自分のGoogl...,# xperia z 2 する スケジュール ストリート 登録 時 いちいち 自分 goog...
5,15,10040,10000,522411417584943104,0,0,0,0,1.0,もはやtablet Pは黒歴史なのかしら・・・？ #xperia #xperiajp\n,もはや tablet p 黒 歴史 だ の # xperia xperiajp
6,18,10043,10000,522411649785806851,0,0,0,1,0.0,Xperia Z3は今月23日発売？\n,xperia z 3 今月 23 日 発売
7,20,10045,10000,522412346459701250,0,0,0,1,0.0,まいった。Xperiaの画面故障でロックが外せない→データバックアップできない。ちくしょーび...,まいる た xperia 画面 故障 ロック 外せる ない データ バックアップ できる ち...
8,21,10046,10000,522412706792353792,0,0,0,1,0.0,親が新型Xperiaの購入を考え始めた( ◜‸‾)\n,親 新型 xperia 購入 考える 始める た ( ◜‸‾)
9,23,10048,10000,522413880534765568,0,0,0,1,0.0,Xperiaほーしー #daitoanime\n,xperia ほる ー する # daitoanime


In [9]:
tweets_vec[0]

array([1, 2, 3, 4, 5, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [10]:
 tweets_y[0]

[0, 0, 0, 1, 0.0]

In [11]:
len(tweets_vec), len(tweets), len(tweets_y)

(51088, 51088, 51088)

In [12]:
tweetid_vec = {}
tweetid_y = {}
for idx, tweet in tweets.iterrows():
    tweetid = tweet['id']
    tweetid_vec[tweetid] = tweets_vec[idx]
    tweetid_y[tweetid] = tweets_y[idx]

# split data

In [13]:
pn = tweets[tweets['pn']==1]
p = tweets[tweets['p']==1]
n = tweets[tweets['n']==1]
neu = tweets[tweets['neu']==1]
unk = tweets[tweets['unk']==1]

In [14]:
len(pn), len(p), len(n), len(neu), len(unk)

(262, 3028, 5174, 31492, 11132)

In [15]:
train_pn = pn.sample(frac=0.8)
test_pn = pn.drop(train_pn.index)

train_p = p.sample(frac=0.8)
test_p = p.drop(train_p.index)

train_n = n.sample(frac=0.8)
test_n = n.drop(train_n.index)

train_neu = neu.sample(frac=0.8)
test_neu = neu.drop(train_neu.index)

train_unk = unk.sample(frac=0.8)
test_unk = unk.drop(train_unk.index)

In [16]:
len(train_unk), len(test_unk)

(8906, 2226)

In [17]:
train = train_pn
train = train.append(train_p)
train = train.append(train_n)
train = train.append(train_neu)
train = train.append(train_unk)

In [18]:
test = test_pn
test = test.append(test_p)
test = test.append(test_n)
test = test.append(test_neu)
test = test.append(test_unk)

In [19]:
len(train), len(test), len(train)+len(test)

(40871, 10217, 51088)

In [20]:
train_vec = []
train_y = []
for _, item in train.iterrows():
    tweet_id = item['id']
    train_vec.append(tweetid_vec[tweet_id])
    train_y.append(tweetid_y[tweet_id])

In [21]:
test_vec = []
test_y = []
for _, item in test.iterrows():
    tweet_id = item['id']
    test_vec.append(tweetid_vec[tweet_id])
    test_y.append(tweetid_y[tweet_id])

In [22]:
len(train_vec), len(train_y), len(test_vec), len(test_y)

(40871, 40871, 10217, 10217)

In [23]:
pickle.dump(train, open('../data/train.pkl', 'wb'))
pickle.dump(train_vec, open('../data/train_vec.pkl', 'wb'))
pickle.dump(train_y, open('../data/train_y.pkl', 'wb'))
pickle.dump(test, open('../data/test.pkl', 'wb'))
pickle.dump(test_vec, open('../data/test_vec.pkl', 'wb'))
pickle.dump(test_y, open('../data/test_y.pkl', 'wb'))
pickle.dump(pre_embedding, open('../data/pre_embedding.pkl', 'wb'))
pickle.dump(vocab_processor, open('../data/vocab_processor.pkl', 'wb'))
pickle.dump(vocab_dict, open('../data/vocab_dict.pkl', 'wb'))